In [57]:
import numpy as np
import torch
from LIM.neural_networks.models import LIM_class
from neural_networks.utilities import *
import cftime

%load_ext autoreload
%autoreload 2

# Create the DataLoader for first principal component
data = torch.load("./neural_networks/data/data_piControl.pt")
print("Data shape : {}".format(data.shape))

data = normalize_data(data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Data shape : torch.Size([30, 14400])


In [58]:
eps = 0.001
timesteps = 120000
time = True
# original fit
tau = 1
model_org = LIM_class.LIM(tau)
model_org.fit(data.detach().cpu().numpy(), eps=eps)
#131999 = 11000 years
lim_integration, times_ = model_org.noise_integration(data[:, -1], timesteps=timesteps, num_comp=30)
lim_integration = lim_integration.T

lim_data = torch.zeros(lim_integration.shape[0]+1, lim_integration.shape[1])

if time is True:
    count = 0
    for t in range(int(timesteps / 12)):
        for m in range(12):
            month = np.array(m+1)
            month = np.expand_dims(month, axis=0)
            lim = np.append(lim_integration[:, count + m], month, axis=0)
            lim_data[:, count+m] = torch.from_numpy(lim)
        count += 12
    lim_integration = lim_data

if time is False:
    lim_integration = torch.from_numpy(lim_integration)
torch.save(lim_integration, "./neural_networks/synthetic_data/lim_integration_TIME_120k[-1]p.pt")
print("Saved data")

Covariance matrix has negative values!
Saved data


In [59]:

eofs = np.arange(0, 30)

date_end = (timesteps / 12) + 12
# Set the start and end dates
start_date = cftime.DatetimeNoLeap(1, 1, 15, 12, 0, 0, 0, has_year_zero=True)
end_date = cftime.DatetimeNoLeap(date_end, 10, 15, 12, 0, 0, 0, has_year_zero=True)

# Create a range of monthly timestamps
time = xr.cftime_range(start=start_date, end=end_date, freq='M')

print("Lim interation : {} {}".format(type(lim_integration), lim_integration.shape))
# Create a DataArray from the numpy array with coordinates
data_xr = xr.DataArray(lim_integration, coords=[np.arange(30), time], dims=['eof', 'time'])

# Print the created xarray
print(type(data_xr))

#data_xr.to_netcdf('./neural_networks/synthetic_data/lim_integration_xarray_20k[-1]p.nc')
print("Save data")

Lim interation : <class 'torch.Tensor'> torch.Size([31, 120000])


ValueError: conflicting sizes for dimension 'eof': length 31 on the data but length 30 on coordinate 'eof'